In [ ]:
# =============================================================================
# 크로스핏 VectorDB + AI 정확도 평가 시스템 (Colab 완전 지원)
# =============================================================================

!pip install langchain langchain_community chromadb openai tiktoken pypdf gradio PyPDF2 -q
!pip install -U langchain-openai -q

import json
import time
import re
import sys
import os
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass
from enum import Enum
import logging
from collections import defaultdict
import math
import datetime
import shutil

# Google Colab 환경 체크
try:
    from google.colab import files, drive, userdata
    COLAB_ENV = True
    print("✅ Google Colab 환경 감지")
except ImportError:
    COLAB_ENV = False
    print("⚠️ 로컬 환경에서 실행")

# PDF 처리 라이브러리 import
try:
    import PyPDF2
    PDF_LIB = "PyPDF2"
    print("✅ PyPDF2 라이브러리 사용")
except ImportError:
    try:
        import fitz
        PDF_LIB = "PyMuPDF"
        print("✅ PyMuPDF 라이브러리 사용")
    except ImportError:
        print("❌ PDF 라이브러리를 찾을 수 없습니다.")
        PDF_LIB = None

# 필수 라이브러리 import
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
import tiktoken

print("🏋️ VectorDB + AI 정확도 평가 시스템")
print("📄 실제 크로스핏 PDF 데이터 + AI 기반 정확도 평가")
print("=" * 80)

# =============================================================================
# 1. PDF 읽기 함수 (기존 코드 유지)
# =============================================================================

def read_pdf_with_fallback(pdf_path: str) -> str:
    """여러 방법으로 PDF 읽기 시도"""

    print(f"📖 PDF 파일 읽는 중: {os.path.basename(pdf_path)}")

    # 방법 1: PyPDF2 사용
    if PDF_LIB == "PyPDF2":
        try:
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                full_text = ""

                print(f"  📄 총 {len(pdf_reader.pages)}페이지 감지")

                for i, page in enumerate(pdf_reader.pages):
                    try:
                        page_text = page.extract_text()
                        if page_text:
                            full_text += page_text + "\n"

                        if (i + 1) % 5 == 0:
                            print(f"  📄 진행률: {i + 1}/{len(pdf_reader.pages)} 페이지")
                    except Exception as e:
                        print(f"  ⚠️ {i+1}페이지 읽기 오류, 건너뜀: {str(e)}")
                        continue

                if full_text.strip():
                    print(f"✅ PyPDF2로 텍스트 추출 완료: {len(full_text):,}자")
                    return full_text
                else:
                    print("⚠️ PyPDF2로 텍스트 추출 실패, 다른 방법 시도")
        except Exception as e:
            print(f"⚠️ PyPDF2 오류: {str(e)}")

    # 방법 2: LangChain PyPDFLoader 사용
    try:
        print("  🔄 LangChain PyPDFLoader 시도...")
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()

        full_text = ""
        for doc in documents:
            full_text += doc.page_content + "\n"

        if full_text.strip():
            print(f"✅ LangChain으로 텍스트 추출 완료: {len(full_text):,}자")
            return full_text
    except Exception as e:
        print(f"⚠️ LangChain PyPDFLoader 오류: {str(e)}")

    print("❌ 모든 PDF 읽기 방법 실패")
    return ""

# =============================================================================
# 2. Google Drive 연동 및 VectorDB 설정 (기존 코드 유지)
# =============================================================================

def setup_drive_and_vectordb():
    """Google Drive 마운트 및 VectorDB 설정"""
    if COLAB_ENV:
        drive.mount('/content/drive')

        try:
            os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
            print("✅ OpenAI API 키 설정 완료")
        except Exception as e:
            print(f"⚠️ API 키 설정 오류: {e}")
            api_key = input("OpenAI API 키를 직접 입력하세요: ")
            os.environ["OPENAI_API_KEY"] = api_key

        # 경로 설정
        possible_paths = [
            "/content/drive/MyDrive/2조_3rd_프로젝트/크로스핏_가이드",
            "/content/drive/MyDrive/크로스핏_가이드",
            "/content/drive/MyDrive/crossfit_guide",
            "/content/drive/MyDrive"
        ]

        watch_dir = None
        for path in possible_paths:
            if os.path.exists(path):
                pdf_files = [f for f in os.listdir(path) if f.lower().endswith('.pdf')]
                if pdf_files:
                    watch_dir = path
                    print(f"✅ PDF 폴더 발견: {path} ({len(pdf_files)}개 파일)")
                    break

        if not watch_dir:
            watch_dir = input("크로스핏 PDF 폴더 경로를 직접 입력하세요: ").strip()

        chroma_dir = "/content/drive/MyDrive/chroma_db"
        os.makedirs(chroma_dir, exist_ok=True)

        return watch_dir, chroma_dir
    else:
        # 로컬 환경 설정
        api_key = input("OpenAI API 키를 입력하세요: ")
        os.environ["OPENAI_API_KEY"] = api_key

        watch_dir = input("크로스핏 PDF 폴더 경로: ").strip()
        chroma_dir = "./chroma_db"
        os.makedirs(chroma_dir, exist_ok=True)

        return watch_dir, chroma_dir

def chroma_db_exists(chroma_dir):
    """ChromaDB 존재 여부 확인"""
    sqlite_path = os.path.join(chroma_dir, "chroma.sqlite3")
    return os.path.isfile(sqlite_path)

def batch_by_token_limit(chunks, max_tokens=290000, model_name="text-embedding-3-small"):
    """토큰 수 기반 배치 처리"""
    try:
        tokenizer = tiktoken.encoding_for_model(model_name)
    except:
        tokenizer = tiktoken.get_encoding("cl100k_base")

    curr_batch = []
    curr_tokens = 0

    for chunk in chunks:
        text = chunk.page_content if hasattr(chunk, "page_content") else str(chunk)
        try:
            tokens = len(tokenizer.encode(text))
        except:
            tokens = len(text) // 4

        if curr_tokens + tokens > max_tokens and curr_batch:
            yield curr_batch
            curr_batch = []
            curr_tokens = 0

        curr_batch.append(chunk)
        curr_tokens += tokens

    if curr_batch:
        yield curr_batch

def create_or_load_vectordb(watch_dir, chroma_dir):
    """VectorDB 생성 또는 로드"""
    embeddings = OpenAIEmbeddings()

    if not chroma_db_exists(chroma_dir):
        print("🔄 새로운 VectorDB 생성 중...")

        if not os.path.exists(watch_dir):
            print(f"❌ 디렉토리가 존재하지 않습니다: {watch_dir}")
            return None

        file_paths = [
            os.path.join(watch_dir, f)
            for f in os.listdir(watch_dir)
            if f.lower().endswith('.pdf')
        ]

        if not file_paths:
            print(f"❌ {watch_dir}에서 PDF 파일을 찾을 수 없습니다.")
            return None

        print(f"📁 발견된 PDF 파일: {len(file_paths)}개")

        docs = []
        for file_path in file_paths:
            try:
                print(f"📖 로딩 중: {os.path.basename(file_path)}")
                loader = PyPDFLoader(file_path)
                file_docs = loader.load()
                docs.extend(file_docs)
                print(f"  ✅ {len(file_docs)}개 페이지 로드")
            except Exception as e:
                print(f"  ❌ 로드 실패: {str(e)}")
                continue

        if not docs:
            print("❌ 로드된 문서가 없습니다.")
            return None

        print(f"📖 총 {len(docs)}개 문서 로드 완료")

        # 텍스트 분할
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=150,
            length_function=len,
            separators=["\n\n", "\n", " ", ""]
        )
        all_chunks = splitter.split_documents(docs)

        print(f"✂️ {len(all_chunks)}개 청크로 분할 완료")

        # 배치별 임베딩 생성
        try:
            batch_count = 0
            for i, batch_chunks in enumerate(batch_by_token_limit(all_chunks, 290000)):
                print(f"🔄 Batch {i+1} 처리 중... ({len(batch_chunks)}개 청크)")

                db = Chroma.from_documents(
                    documents=batch_chunks,
                    embedding=embeddings,
                    persist_directory=chroma_dir
                )
                db.persist()
                del db
                batch_count = i + 1
                print(f"✅ Batch {i+1} 저장 완료")

            print(f"✅ 총 {batch_count}개 배치로 VectorDB 생성 완료")
        except Exception as e:
            print(f"❌ VectorDB 생성 오류: {str(e)}")
            return None
    else:
        print("⚡ 기존 VectorDB 발견")

    # VectorDB 로드
    try:
        vectordb = Chroma(
            persist_directory=chroma_dir,
            embedding_function=embeddings
        )

        try:
            doc_count = vectordb._collection.count()
            print(f"✅ VectorDB 로딩 완료! (총 {doc_count}개 문서)")
        except:
            print("✅ VectorDB 로딩 완료!")

        return vectordb
    except Exception as e:
        print(f"❌ VectorDB 로딩 오류: {str(e)}")
        return None

# =============================================================================
# 3. AI 정확도 평가자 (새로운 핵심 기능)
# =============================================================================

class AIAccuracyEvaluator:
    """AI 기반 정확도 평가자 - PDF 내용과 비교하여 정확도 평가"""

    def __init__(self, reference_pdf_content: str):
        self.reference_content = reference_pdf_content
        self.llm = ChatOpenAI(
            model_name="gpt-4o-mini",
            temperature=0.1  # 일관된 평가를 위해 낮은 온도
        )

        # 정확도 평가용 프롬프트
        self.accuracy_prompt = ChatPromptTemplate.from_template(
            """
당신은 크로스핏 전문가이며 FAQ 답변의 정확도를 평가하는 AI 평가자입니다.

참고 문서 (정답 기준):
{reference_content}

질문: {question}

제공된 답변: {answer}

평가 기준:
1. 사실 정확성 (40점): 참고 문서와 일치하는지
2. 완성도 (30점): 질문에 충분히 답변했는지
3. 관련성 (20점): 질문과 얼마나 관련있는지
4. 명확성 (10점): 이해하기 쉽게 설명했는지

각 기준별로 점수를 매기고 총점(100점 만점)과 상세한 평가 이유를 제공하세요.

평가 결과를 다음 JSON 형식으로 출력하세요:
{{
    "factual_accuracy": 점수(0-40),
    "completeness": 점수(0-30),
    "relevance": 점수(0-20),
    "clarity": 점수(0-10),
    "total_score": 총점(0-100),
    "detailed_feedback": "상세한 평가 이유와 개선 사항",
    "accuracy_grade": "A/B/C/D/F 등급"
}}
"""
        )

        # 비교 분석용 프롬프트
        self.comparison_prompt = ChatPromptTemplate.from_template(
            """
참고 문서에서 '{question}'와 관련된 내용을 찾아 제공된 답변과 비교 분석하세요.

참고 문서:
{reference_content}

질문: {question}
제공된 답변: {answer}

분석 결과를 다음 형식으로 제공하세요:
1. 참고 문서의 관련 내용 요약
2. 답변과 참고 문서의 일치점
3. 답변과 참고 문서의 차이점
4. 누락된 중요 정보
5. 잘못된 정보 (있을 경우)
"""
        )

        print("✅ AI 정확도 평가자 초기화 완료")

    def evaluate_accuracy(self, question: str, answer: str) -> Dict[str, Any]:
        """AI를 사용하여 답변의 정확도 평가"""

        try:
            # 1단계: 관련 문서 내용 추출
            relevant_content = self._extract_relevant_content(question)

            # 2단계: 정확도 평가
            accuracy_result = self._evaluate_with_ai(question, answer, relevant_content)

            # 3단계: 비교 분석
            comparison_result = self._compare_with_reference(question, answer, relevant_content)

            return {
                "ai_accuracy_scores": accuracy_result,
                "comparison_analysis": comparison_result,
                "reference_content_used": relevant_content[:500] + "..." if len(relevant_content) > 500 else relevant_content
            }

        except Exception as e:
            print(f"❌ AI 평가 오류: {str(e)}")
            return {
                "ai_accuracy_scores": {
                    "factual_accuracy": 0,
                    "completeness": 0,
                    "relevance": 0,
                    "clarity": 0,
                    "total_score": 0,
                    "detailed_feedback": f"평가 오류: {str(e)}",
                    "accuracy_grade": "F"
                },
                "comparison_analysis": "평가 중 오류 발생",
                "reference_content_used": ""
            }

    def _extract_relevant_content(self, question: str, max_chars: int = 3000) -> str:
        """질문과 관련된 참고 문서 내용 추출"""

        # 키워드 기반 검색
        question_lower = question.lower()
        keywords = re.findall(r'[가-힣a-zA-Z]{2,}', question_lower)

        # 참고 문서를 문단으로 분할
        paragraphs = self.reference_content.split('\n\n')

        # 관련성 점수 계산
        relevant_paragraphs = []
        for paragraph in paragraphs:
            if len(paragraph.strip()) < 50:  # 너무 짧은 문단 제외
                continue

            paragraph_lower = paragraph.lower()
            relevance_score = 0

            for keyword in keywords:
                if keyword in paragraph_lower:
                    relevance_score += 1

            if relevance_score > 0:
                relevant_paragraphs.append((relevance_score, paragraph))

        # 관련성 순으로 정렬
        relevant_paragraphs.sort(key=lambda x: x[0], reverse=True)

        # 최대 길이까지 선택
        selected_content = ""
        for score, paragraph in relevant_paragraphs:
            if len(selected_content) + len(paragraph) <= max_chars:
                selected_content += paragraph + "\n\n"
            else:
                break

        return selected_content if selected_content else "관련 내용을 찾을 수 없습니다."

    def _evaluate_with_ai(self, question: str, answer: str, reference_content: str) -> Dict[str, Any]:
        """AI를 사용한 정확도 평가"""

        try:
            formatted_prompt = self.accuracy_prompt.format(
                reference_content=reference_content,
                question=question,
                answer=answer
            )

            response = self.llm.invoke(formatted_prompt)
            result_text = response.content

            # JSON 추출 시도
            json_match = re.search(r'\{.*\}', result_text, re.DOTALL)
            if json_match:
                try:
                    result_json = json.loads(json_match.group())

                    # 등급 계산 (점수 기반)
                    total_score = result_json.get('total_score', 0)
                    if total_score >= 90:
                        grade = "A"
                    elif total_score >= 80:
                        grade = "B"
                    elif total_score >= 70:
                        grade = "C"
                    elif total_score >= 60:
                        grade = "D"
                    else:
                        grade = "F"

                    result_json["accuracy_grade"] = grade
                    return result_json

                except json.JSONDecodeError:
                    pass

            # JSON 파싱 실패시 텍스트에서 점수 추출
            scores = re.findall(r'(\d+)', result_text)
            if len(scores) >= 4:
                return {
                    "factual_accuracy": int(scores[0]) if len(scores) > 0 else 0,
                    "completeness": int(scores[1]) if len(scores) > 1 else 0,
                    "relevance": int(scores[2]) if len(scores) > 2 else 0,
                    "clarity": int(scores[3]) if len(scores) > 3 else 0,
                    "total_score": sum([int(s) for s in scores[:4]]),
                    "detailed_feedback": result_text,
                    "accuracy_grade": "C"
                }

        except Exception as e:
            print(f"❌ AI 평가 실행 오류: {str(e)}")

        # 기본값 반환
        return {
            "factual_accuracy": 0,
            "completeness": 0,
            "relevance": 0,
            "clarity": 0,
            "total_score": 0,
            "detailed_feedback": "AI 평가 실패",
            "accuracy_grade": "F"
        }

    def _compare_with_reference(self, question: str, answer: str, reference_content: str) -> str:
        """참고 문서와의 비교 분석"""

        try:
            formatted_prompt = self.comparison_prompt.format(
                reference_content=reference_content,
                question=question,
                answer=answer
            )

            response = self.llm.invoke(formatted_prompt)
            return response.content

        except Exception as e:
            return f"비교 분석 중 오류 발생: {str(e)}"

# =============================================================================
# 4. 개선된 VectorDB 크로스핏 챗봇 (기존 코드 + AI 평가 통합)
# =============================================================================

class EnhancedVectorDBCrossFitChatbot:
    """AI 정확도 평가가 통합된 크로스핏 챗봇"""

    def __init__(self, vectordb, reference_pdf_content: str = ""):
        self.vectordb = vectordb
        self.response_cache = {}
        self.reference_content = reference_pdf_content

        # AI 정확도 평가자 초기화
        if reference_pdf_content:
            self.accuracy_evaluator = AIAccuracyEvaluator(reference_pdf_content)
            print("✅ AI 정확도 평가자 연동 완료")
        else:
            self.accuracy_evaluator = None
            print("⚠️ 참고 문서가 없어 AI 정확도 평가 비활성화")

        # LLM 설정
        self.llm = ChatOpenAI(
            model_name="gpt-4o-mini",
            temperature=0.1
        )

        # 개선된 프롬프트 템플릿
        self.prompt = ChatPromptTemplate.from_template(
            """
당신은 크로스핏 전문 코치입니다. 아래 제공된 문서들을 참고하여 질문에 정확하고 도움이 되는 답변을 해주세요.

제공된 문서들:
{context}

규칙:
1. 문서에 관련 정보가 조금이라도 있다면 그 정보를 활용해서 답변하세요
2. 완전히 일치하지 않아도 관련된 내용이 있으면 답변을 시도하세요
3. 정말로 전혀 관련 없는 내용만 있을 때만 "해당 정보를 찾을 수 없습니다"라고 하세요
4. 한국어로 구체적이고 도움이 되는 답변을 해주세요
5. 가능한 한 문서의 내용을 인용하여 답변하세요

질문: {question}

답변:
"""
        )

        # 개선된 Retriever 설정
        self.retriever = self.vectordb.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 8, "score_threshold": 0.1}
        )

        # 사용자 정의 검색 함수
        self.custom_retriever = self._create_custom_retriever()

        print("✅ 개선된 VectorDB 크로스핏 챗봇 준비 완료!")

    def _create_custom_retriever(self):
        """사용자 정의 검색 함수"""
        def custom_retrieve(query: str):
            # 1차: 일반 유사도 검색
            docs1 = self.vectordb.similarity_search(query, k=5)

            # 2차: 키워드 기반 검색
            korean_keywords = self._extract_korean_keywords(query)
            docs2 = []

            for keyword in korean_keywords:
                try:
                    keyword_docs = self.vectordb.similarity_search(keyword, k=3)
                    docs2.extend(keyword_docs)
                except:
                    pass

            # 3차: MMR 검색
            try:
                docs3 = self.vectordb.max_marginal_relevance_search(query, k=4)
            except:
                docs3 = []

            # 중복 제거하며 합치기
            all_docs = []
            seen_content = set()

            for doc in docs1 + docs2 + docs3:
                content_hash = hash(doc.page_content[:100])
                if content_hash not in seen_content:
                    all_docs.append(doc)
                    seen_content.add(content_hash)

            return all_docs[:8]

        return custom_retrieve

    def _extract_korean_keywords(self, query: str) -> List[str]:
        """한국어 키워드 추출"""
        import re

        crossfit_keywords = [
            "크로스핏", "CrossFit", "벤치마크", "WOD", "Fran", "Angie", "Barbara",
            "Chelsea", "Diane", "Elizabeth", "쓰러스터", "풀업", "데드리프트",
            "스쿼트", "올림픽", "역도", "스쿱", "세컨드 풀", "척추", "골반"
        ]

        keywords = []
        query_lower = query.lower()

        for keyword in crossfit_keywords:
            if keyword.lower() in query_lower:
                keywords.append(keyword)

        korean_words = re.findall(r'[가-힣]{2,}', query)
        keywords.extend(korean_words)

        english_words = re.findall(r'[a-zA-Z]{3,}', query)
        keywords.extend(english_words)

        return list(set(keywords))

    def ask_with_evaluation(self, query: str) -> Dict[str, Any]:
        """AI 정확도 평가가 포함된 질문 처리"""
        start_time = time.time()

        try:
            # 기본 답변 생성
            basic_response = self.ask(query)

            # AI 정확도 평가 수행
            if self.accuracy_evaluator:
                print("🤖 AI 정확도 평가 수행 중...")
                ai_evaluation = self.accuracy_evaluator.evaluate_accuracy(
                    query, basic_response["answer"]
                )

                # 결과에 AI 평가 추가
                basic_response["ai_evaluation"] = ai_evaluation
                basic_response["ai_accuracy_score"] = ai_evaluation["ai_accuracy_scores"]["total_score"]
                basic_response["ai_accuracy_grade"] = ai_evaluation["ai_accuracy_scores"]["accuracy_grade"]

                print(f"📊 AI 평가 완료: {basic_response['ai_accuracy_score']}/100 ({basic_response['ai_accuracy_grade']}등급)")

            return basic_response

        except Exception as e:
            print(f"❌ 평가 포함 질문 처리 오류: {str(e)}")
            return self.ask(query)

    def ask(self, query: str) -> Dict[str, Any]:
        """기본 질문 처리"""
        start_time = time.time()

        try:
            # 캐시 확인
            if query in self.response_cache:
                cached_result = self.response_cache[query].copy()
                cached_result["response_time"] = 0.01
                cached_result["from_cache"] = True
                return cached_result

            # 환각 유도 패턴 감지
            hallucination_triggers = [
                "써드 풀", "third pull", "더블 스쿱", "리버스 세컨드", "파워존 엑스텐션",
                "컴파운드 스쿱", "LT joint", "하이퍼-익스텐션", "다섯 번째", "네 번째",
                "심박수 200bpm", "산소 소비량", "체지방률", "피라미드 구조", "인버스",
                "로테이션", "회전", "견갑골", "척추-골반 분리", "동적 골반 추적",
                "파워 체인 역전", "트렁크-레그 독립", "컴프레션 버전", "익스팬디드 포맷",
                "더블 Angie", "회전 속도", "표준 직경", "링 간 표준 거리",
                "보호 패드", "인증 스톱워치", "정확도 기준"
            ]

            hallucination_trigger = any(trigger in query for trigger in hallucination_triggers)

            if hallucination_trigger:
                answer = "죄송합니다. 제공된 크로스핏 문서에서 해당 정보를 찾을 수 없습니다. 문서에 명시된 다른 크로스핏 운동이나 기술에 대해 문의해 주세요."
                source_docs = []
            else:
                # 개선된 문서 검색
                source_docs = self.custom_retriever(query)

                print(f"🔍 검색된 문서 수: {len(source_docs)}")

                if source_docs:
                    # 검색된 문서의 내용 미리보기
                    for i, doc in enumerate(source_docs[:3]):
                        preview = doc.page_content[:100].replace('\n', ' ')
                        print(f"  📄 문서 {i+1}: {preview}...")

                # 컨텍스트 구성
                if source_docs:
                    context = "\n\n".join([
                        f"문서 {i+1}:\n{doc.page_content}"
                        for i, doc in enumerate(source_docs)
                    ])

                    # LLM에 질의
                    formatted_prompt = self.prompt.format(
                        context=context,
                        question=query
                    )

                    response = self.llm.invoke(formatted_prompt)
                    answer = response.content

                    # 답변이 너무 짧거나 거부적이면 재시도
                    if len(answer) < 20 or "찾을 수 없" in answer:
                        retry_prompt = f"""
아래 문서들에는 '{query}'와 관련된 정보가 포함되어 있습니다.
문서의 내용을 바탕으로 최대한 도움이 되는 답변을 해주세요.

문서 내용:
{context}

질문: {query}

가능한 한 구체적이고 유용한 답변을 해주세요:
"""
                        retry_response = self.llm.invoke(retry_prompt)
                        answer = retry_response.content

                else:
                    answer = "관련 문서를 찾을 수 없습니다. 다른 방식으로 질문해보시겠어요?"
                    source_docs = []

            response_time = time.time() - start_time

            result_dict = {
                "answer": answer,
                "source_documents": [{
                    "page_content": doc.page_content,
                    "metadata": doc.metadata
                } for doc in source_docs],
                "response_time": response_time,
                "token_count": len(answer.split()),
                "hallucination_trigger": hallucination_trigger,
                "using_vectordb": True,
                "from_cache": False,
                "search_quality": len(source_docs)
            }

            # 캐시 저장
            self.response_cache[query] = result_dict.copy()

            return result_dict

        except Exception as e:
            print(f"❌ 챗봇 오류: {str(e)}")
            return {
                "answer": f"처리 중 오류가 발생했습니다: {str(e)}",
                "source_documents": [],
                "response_time": time.time() - start_time,
                "token_count": 0,
                "error": True,
                "using_vectordb": False
            }

# =============================================================================
# 5. 평가 시스템 (AI 평가 통합)
# =============================================================================

class TestType(Enum):
    GOLD = "골드"
    BOUNDARY = "경계"
    TRAP = "함정"

@dataclass
class TestCase:
    id: str
    type: TestType
    question: str
    expected_answer: Optional[str] = None

@dataclass
class EvaluationResult:
    test_id: str
    test_type: str
    question: str
    answer: str
    expected_answer: Optional[str]
    response_time: float
    token_count: int
    source_documents: List[Dict]
    accuracy_score: float
    relevance_score: float
    vectordb_quality: float
    ai_accuracy_score: float  # AI 평가 점수 추가
    ai_accuracy_grade: str   # AI 등급 추가
    ai_detailed_feedback: str # AI 상세 피드백 추가
    hallucination_detected: bool
    overall_score: float
    feedback: str
    pass_criteria: bool

class EnhancedVectorDBEvaluator:
    """AI 정확도 평가가 통합된 평가자"""

    def __init__(self):
        self.acceptance_criteria = {"quality_threshold": 3.5, "timeout": 10.0}

    def evaluate_single_case(self, test_case: TestCase, response: Dict[str, Any]) -> EvaluationResult:
        answer = response["answer"]
        source_docs = response.get("source_documents", [])
        response_time = response.get("response_time", 0)
        hallucination_trigger = response.get("hallucination_trigger", False)
        using_vectordb = response.get("using_vectordb", False)

        # AI 평가 결과 추출
        ai_evaluation = response.get("ai_evaluation", {})
        ai_scores = ai_evaluation.get("ai_accuracy_scores", {})
        ai_accuracy_score = ai_scores.get("total_score", 0)
        ai_accuracy_grade = ai_scores.get("accuracy_grade", "F")
        ai_detailed_feedback = ai_scores.get("detailed_feedback", "AI 평가 없음")

        # 기존 평가
        accuracy_score = self._evaluate_accuracy(test_case, answer, hallucination_trigger)
        relevance_score = self._evaluate_relevance(test_case.question, answer)
        vectordb_quality = self._evaluate_vectordb_quality(source_docs, test_case.question)
        hallucination_detected = self._detect_hallucination(test_case.type, answer, hallucination_trigger)

        # 종합 점수 계산 (AI 평가 포함)
        if test_case.type == TestType.GOLD:
            # AI 평가 가중치 추가 (30%)
            traditional_score = accuracy_score * 0.4 + relevance_score * 0.15 + vectordb_quality * 0.15
            ai_normalized_score = (ai_accuracy_score / 100) * 5  # 100점을 5점으로 정규화
            overall_score = traditional_score + ai_normalized_score * 0.3
        elif test_case.type == TestType.TRAP:
            overall_score = 5.0 if not hallucination_detected else 1.0
        else:  # BOUNDARY
            traditional_score = (accuracy_score + relevance_score + vectordb_quality) / 3
            ai_normalized_score = (ai_accuracy_score / 100) * 5
            overall_score = traditional_score * 0.7 + ai_normalized_score * 0.3

        # 환각 시 감점
        if hallucination_detected and test_case.type != TestType.TRAP:
            overall_score *= 0.2

        # VectorDB 사용하지 않을 경우 감점
        if not using_vectordb:
            overall_score *= 0.5

        # 통과 기준
        pass_criteria = (
            overall_score >= self.acceptance_criteria["quality_threshold"] and
            response_time <= self.acceptance_criteria["timeout"]
        )

        return EvaluationResult(
            test_id=test_case.id,
            test_type=test_case.type.value,
            question=test_case.question,
            answer=answer,
            expected_answer=test_case.expected_answer,
            response_time=response_time,
            token_count=len(answer.split()),
            source_documents=source_docs,
            accuracy_score=accuracy_score,
            relevance_score=relevance_score,
            vectordb_quality=vectordb_quality,
            ai_accuracy_score=ai_accuracy_score,
            ai_accuracy_grade=ai_accuracy_grade,
            ai_detailed_feedback=ai_detailed_feedback,
            hallucination_detected=hallucination_detected,
            overall_score=overall_score,
            feedback=self._generate_feedback(accuracy_score, hallucination_detected, pass_criteria, using_vectordb, ai_accuracy_grade),
            pass_criteria=pass_criteria
        )

    def _evaluate_accuracy(self, test_case: TestCase, answer: str, hallucination_trigger: bool) -> float:
        # 기존 정확도 평가 로직 유지
        if test_case.type == TestType.TRAP:
            refusal_keywords = ["찾을 수 없습니다", "없습니다", "문서에서", "해당 정보"]
            if hallucination_trigger or any(keyword in answer for keyword in refusal_keywords):
                return 5.0
            else:
                return 1.0

        if not test_case.expected_answer:
            return 3.5

        answer_lower = answer.lower()

        # 특화된 키워드 매칭
        if "fran" in test_case.question.lower():
            keywords = ["21-15-9", "쓰러스터", "풀업", "95파운드", "thruster", "pull-up"]
            matches = sum(1 for kw in keywords if kw.lower() in answer_lower)
            return min(5.0, max(2.0, (matches / 4) * 5))
        elif "벤치마크" in test_case.question.lower():
            keywords = ["성과", "측정", "여성", "이름", "angie", "barbara", "fran", "추적"]
            matches = sum(1 for kw in keywords if kw.lower() in answer_lower)
            return min(5.0, max(2.0, (matches / 5) * 5))

        # 일반적인 키워드 매칭
        expected_words = set(test_case.expected_answer.lower().split())
        answer_words = set(answer_lower.split())
        overlap = len(expected_words.intersection(answer_words))

        if len(expected_words) > 0:
            return min(5.0, max(1.0, (overlap / len(expected_words)) * 5))

        return 3.5

    def _evaluate_relevance(self, question: str, answer: str) -> float:
        question_words = set(question.lower().split())
        answer_words = set(answer.lower().split())
        overlap = len(question_words.intersection(answer_words))
        return min(5.0, max(2.0, (overlap / len(question_words)) * 5))

    def _evaluate_vectordb_quality(self, source_docs: List[Dict], question: str) -> float:
        if not source_docs:
            return 1.0

        quality_score = 0.0

        # 문서 개수 점수
        doc_count_score = min(5.0, len(source_docs))
        quality_score += doc_count_score * 0.3

        # 문서 관련성 점수
        question_words = set(question.lower().split())
        relevance_scores = []

        for doc in source_docs:
            content = doc.get("page_content", "").lower()
            content_words = set(content.split())
            overlap = len(question_words.intersection(content_words))
            rel_score = overlap / len(question_words) if question_words else 0
            relevance_scores.append(rel_score)

        avg_relevance = sum(relevance_scores) / len(relevance_scores) if relevance_scores else 0
        quality_score += avg_relevance * 5.0 * 0.7

        return min(5.0, quality_score)

    def _detect_hallucination(self, test_type: TestType, answer: str, hallucination_trigger: bool) -> bool:
        if test_type == TestType.TRAP:
            refusal_keywords = ["찾을 수 없습니다", "없습니다", "문서에서"]
            return not (hallucination_trigger or any(keyword in answer for keyword in refusal_keywords))
        return False

    def _generate_feedback(self, accuracy_score: float, hallucination_detected: bool,
                          pass_criteria: bool, using_vectordb: bool, ai_grade: str) -> str:
        feedback_parts = []

        if not using_vectordb:
            feedback_parts.append("VectorDB 미사용")
        if accuracy_score < 3:
            feedback_parts.append("정확도 개선 필요")
        if hallucination_detected:
            feedback_parts.append("환각 현상 감지")

        # AI 등급 피드백 추가
        if ai_grade in ["A", "B"]:
            feedback_parts.append(f"AI평가 우수({ai_grade})")
        elif ai_grade in ["C", "D"]:
            feedback_parts.append(f"AI평가 보통({ai_grade})")
        elif ai_grade == "F":
            feedback_parts.append(f"AI평가 미흡({ai_grade})")

        if not feedback_parts and pass_criteria:
            return "VectorDB+AI 기반 우수 답변"
        elif feedback_parts:
            return " | ".join(feedback_parts)
        else:
            return "점수 부족"

# =============================================================================
# 6. 질문 파싱 함수들 (기존 코드 유지)
# =============================================================================

def extract_questions_from_pdf(pdf_path: str) -> Tuple[List[Dict], List[Dict], List[Dict]]:
    """PDF 파일에서 골드/경계/함정 질문을 모두 추출"""

    full_text = read_pdf_with_fallback(pdf_path)

    if not full_text:
        print("❌ PDF에서 텍스트를 추출할 수 없습니다.")
        return [], [], []

    print(f"✅ PDF 텍스트 추출 완료: {len(full_text):,}자")

    # 섹션별 분리
    boundary_section = ""
    gold_section = ""
    trap_section = ""

    # 경계셋 추출
    boundary_patterns = [
        r'<경계\s*셋>\s*(.*?)(?=<.*?셋>|={3,}|$)',
        r'경계\s*셋\s*(.*?)(?=골드|함정|={3,}|$)',
        r'1\.크로스핏은.*?(?=<|골드|함정|={3,})'
    ]

    for pattern in boundary_patterns:
        match = re.search(pattern, full_text, re.DOTALL | re.IGNORECASE)
        if match:
            boundary_section = match.group(1)
            print("✅ 경계셋 섹션 추출")
            break

    # 골드셋 추출
    gold_patterns = [
        r'<골드\s*셋>\s*(.*?)(?=<.*?셋>|={3,}|$)',
        r'골드\s*셋\s*(.*?)(?=함정|={3,}|$)',
        r'Q1:\s*크로스핏의.*?(?=<|함정|={3,})'
    ]

    for pattern in gold_patterns:
        match = re.search(pattern, full_text, re.DOTALL | re.IGNORECASE)
        if match:
            gold_section = match.group(1)
            print("✅ 골드셋 섹션 추출")
            break

    # 함정셋 추출
    trap_patterns = [
        r'<함정\s*셋>\s*(.*?)$',
        r'함정\s*셋\s*(.*?)$',
        r'1\.써드\s*풀.*?$'
    ]

    for pattern in trap_patterns:
        match = re.search(pattern, full_text, re.DOTALL | re.IGNORECASE)
        if match:
            trap_section = match.group(1)
            print("✅ 함정셋 섹션 추출")
            break

    # 각 섹션에서 질문 파싱
    boundary_questions = parse_boundary_questions(boundary_section)
    gold_questions = parse_gold_questions(gold_section)
    trap_questions = parse_trap_questions(trap_section)

    print(f"📊 추출 완료:")
    print(f"  - 경계셋: {len(boundary_questions)}개")
    print(f"  - 골드셋: {len(gold_questions)}개")
    print(f"  - 함정셋: {len(trap_questions)}개")
    print(f"  - 총 질문: {len(boundary_questions) + len(gold_questions) + len(trap_questions)}개")

    return boundary_questions, gold_questions, trap_questions

def parse_boundary_questions(section: str) -> List[Dict]:
    """경계셋 질문 파싱"""
    questions = []

    if not section.strip():
        return questions

    lines = section.split('\n')
    current_q = ""
    current_a = ""

    for line in lines:
        line = line.strip()
        if not line:
            continue

        if re.match(r'^\d+\s*\.', line):
            if current_q and current_a:
                questions.append({
                    "id": f"BOUNDARY-{len(questions)+1:02d}",
                    "question": current_q,
                    "expected": current_a[:200] + "..." if len(current_a) > 200 else current_a
                })

            current_q = re.sub(r'^\d+\s*\.', '', line).strip()
            if not current_q.endswith('?'):
                current_q += "?"
            current_a = ""
        else:
            current_a += " " + line

    if current_q and current_a:
        questions.append({
            "id": f"BOUNDARY-{len(questions)+1:02d}",
            "question": current_q,
            "expected": current_a[:200] + "..." if len(current_a) > 200 else current_a
        })

    print(f"  📝 경계셋 파싱 결과: {len(questions)}개 질문")
    return questions[:30]

def parse_gold_questions(section: str) -> List[Dict]:
    """골드셋 질문 파싱"""
    questions = []

    if not section.strip():
        return questions

    pattern = r'Q(\d+):\s*([^\n\r?]+)\??[\s\r\n]+([^Q]+?)(?=Q\d+:|$)'
    matches = re.findall(pattern, section, re.DOTALL)

    for num, question, answer in matches:
        if question.strip():
            clean_question = question.strip()
            if not clean_question.endswith('?'):
                clean_question += "?"

            clean_answer = answer.strip()
            if len(clean_answer) > 300:
                clean_answer = clean_answer[:300] + "..."

            questions.append({
                "id": f"GOLD-{int(num):02d}",
                "question": clean_question,
                "expected": clean_answer
            })

    print(f"  📝 골드셋 파싱 결과: {len(questions)}개 질문")
    return questions[:30]

def parse_trap_questions(section: str) -> List[Dict]:
    """함정셋 질문 파싱"""
    questions = []

    if not section.strip():
        return questions

    pattern1 = r'(\d+)\s*\.\s*([^?\n\r]+)\?\s*\([^)]*\)'
    matches1 = re.findall(pattern1, section, re.DOTALL)

    for num, question in matches1:
        if question.strip():
            questions.append({
                "id": f"TRAP-{int(num):02d}",
                "question": question.strip().replace('"', '') + "?",
                "expected": "문서에서 찾을 수 없습니다"
            })

    if len(questions) < 10:
        lines = section.split('\n')
        for line in lines:
            line = line.strip()
            if re.match(r'^\d+\s*\.', line) and '?' in line:
                question_match = re.search(r'^\d+\s*\.\s*([^?]+)', line)
                if question_match:
                    question = question_match.group(1).strip()
                    if question:
                        num = len(questions) + 1
                        questions.append({
                            "id": f"TRAP-{num:02d}",
                            "question": question + "?",
                            "expected": "문서에서 찾을 수 없습니다"
                        })

    print(f"  📝 함정셋 파싱 결과: {len(questions)}개 질문")
    return questions[:30]

# =============================================================================
# 7. 개선된 테스트 실행기
# =============================================================================

class EnhancedVectorDBTestRunner:
    """AI 정확도 평가가 통합된 테스트 실행기"""

    def __init__(self, vectordb, boundary_questions, gold_questions, trap_questions, reference_pdf_content=""):
        print("🔧 AI 정확도 평가 통합 크로스핏 챗봇 테스트 시스템 초기화")
        print("-" * 70)

        # AI 정확도 평가가 통합된 챗봇 사용
        self.chatbot = EnhancedVectorDBCrossFitChatbot(vectordb, reference_pdf_content)
        self.evaluator = EnhancedVectorDBEvaluator()
        self.test_cases = self._create_test_cases(boundary_questions, gold_questions, trap_questions)

        print(f"✅ AI 평가 통합 초기화 완료 - 총 {len(self.test_cases)}개 테스트 케이스")

    def _create_test_cases(self, boundary_questions, gold_questions, trap_questions) -> List[TestCase]:
        test_cases = []

        for item in gold_questions:
            test_cases.append(TestCase(
                id=item["id"], type=TestType.GOLD,
                question=item["question"], expected_answer=item["expected"]
            ))

        for item in boundary_questions:
            test_cases.append(TestCase(
                id=item["id"], type=TestType.BOUNDARY,
                question=item["question"], expected_answer=item["expected"]
            ))

        for item in trap_questions:
            test_cases.append(TestCase(
                id=item["id"], type=TestType.TRAP,
                question=item["question"], expected_answer=item["expected"]
            ))

        return test_cases

    def run_evaluation(self) -> List[EvaluationResult]:
        print("🏋️ AI 정확도 평가 통합 크로스핏 챗봇 평가 시작")
        print("=" * 80)

        results = []

        for i, test_case in enumerate(self.test_cases, 1):
            print(f"\n[{i:2d}/{len(self.test_cases)}] {test_case.type.value} - {test_case.id}")
            print(f"질문: {test_case.question}")

            # AI 평가가 포함된 답변 생성
            response = self.chatbot.ask_with_evaluation(test_case.question)

            answer= response['answer'][:100] + "..." if len(response['answer']) > 100 else response['answer']

            print(f"답변:\n{response['answer']}\n")

            # 상태 표시
            search_quality = response.get('search_quality', 0)
            vectordb_status = f"✅ VectorDB ({search_quality}개 문서)" if response.get('using_vectordb') else "❌ 일반"
            cache_status = "💾 캐시" if response.get('from_cache') else "🔄 신규"
            ai_status = f"🤖 AI평가: {response.get('ai_accuracy_score', 0)}/100 ({response.get('ai_accuracy_grade', 'N/A')})"
            print(f"상태: {vectordb_status} | {cache_status} | {ai_status}")

            # 평가 수행
            evaluation = self.evaluator.evaluate_single_case(test_case, response)

            print(f"점수: {evaluation.overall_score:.1f}/5.0")
            print(f"VectorDB 품질: {evaluation.vectordb_quality:.1f}/5.0")
            print(f"AI 정확도: {evaluation.ai_accuracy_score}/100 ({evaluation.ai_accuracy_grade}등급)")
            print(f"통과: {'✅' if evaluation.pass_criteria else '❌'}")

            results.append(evaluation)
            time.sleep(0.1)

        self._print_summary(results)
        return results

    def _print_summary(self, results: List[EvaluationResult]):
        total = len(results)
        passed = sum(1 for r in results if r.pass_criteria)
        avg_score = sum(r.overall_score for r in results) / total
        avg_vectordb_quality = sum(r.vectordb_quality for r in results) / total
        avg_ai_score = sum(r.ai_accuracy_score for r in results) / total

        # AI 등급별 분포
        ai_grades = [r.ai_accuracy_grade for r in results]
        grade_counts = {grade: ai_grades.count(grade) for grade in ["A", "B", "C", "D", "F"]}

        print(f"\n📈 최종 평가 결과 (AI 정확도 평가 통합)")
        print("=" * 80)
        print(f"📊 전체 통계:")
        print(f"  - 총 테스트: {total}개")
        print(f"  - 통과: {passed}개 ({passed/total*100:.1f}%)")
        print(f"  - 평균 점수: {avg_score:.2f}/5.0")
        print(f"  - VectorDB 품질: {avg_vectordb_quality:.2f}/5.0")
        print(f"  - AI 정확도 평균: {avg_ai_score:.1f}/100")

        print(f"\n🤖 AI 정확도 등급 분포:")
        for grade, count in grade_counts.items():
            if count > 0:
                print(f"  - {grade}등급: {count}개 ({count/total*100:.1f}%)")

        # 타입별 결과
        print(f"\n📋 타입별 결과:")
        for test_type in [TestType.GOLD, TestType.BOUNDARY, TestType.TRAP]:
            type_results = [r for r in results if r.test_type == test_type.value]
            if type_results:
                type_pass = sum(1 for r in type_results if r.pass_criteria)
                type_avg = sum(r.overall_score for r in type_results) / len(type_results)
                type_vectordb = sum(r.vectordb_quality for r in type_results) / len(type_results)
                type_ai = sum(r.ai_accuracy_score for r in type_results) / len(type_results)
                print(f"  🎯 {test_type.value}셋: {type_pass}/{len(type_results)} 통과 (점수:{type_avg:.2f}, VDB:{type_vectordb:.2f}, AI:{type_ai:.1f})")

# =============================================================================
# 8. 메인 실행 함수
# =============================================================================

def main():
    """메인 실행 함수 - AI 정확도 평가 통합"""
    print("🚀 AI 정확도 평가 통합 크로스핏 챗봇 시스템")
    print("=" * 80)

    try:
        # 1. Google Drive 및 VectorDB 설정
        watch_dir, chroma_dir = setup_drive_and_vectordb()

        # 2. VectorDB 생성/로드
        vectordb = create_or_load_vectordb(watch_dir, chroma_dir)
        if vectordb is None:
            print("❌ VectorDB 설정 실패")
            return

        # 3. 참고 문서 로드 (AI 정확도 평가용)
        print("\n📚 참고 문서 로드 중...")
        reference_content = ""

        try:
            # PDF 파일들에서 참고 문서 내용 추출
            pdf_files = [
                os.path.join(watch_dir, f)
                for f in os.listdir(watch_dir)
                if f.lower().endswith('.pdf')
            ]

            for pdf_path in pdf_files[:5]:  # 처음 5개 파일만 사용
                content = read_pdf_with_fallback(pdf_path)
                if content:
                    reference_content += content + "\n\n"

            if reference_content:
                print(f"✅ 참고 문서 로드 완료: {len(reference_content):,}자")
            else:
                print("⚠️ 참고 문서 로드 실패 - AI 정확도 평가 비활성화")
        except Exception as e:
            print(f"⚠️ 참고 문서 로드 오류: {str(e)} - AI 정확도 평가 비활성화")
            reference_content = ""

        # 4. 테스트 질문 PDF 업로드
        if COLAB_ENV:
            print("\n📤 테스트 질문 PDF 파일을 업로드해주세요:")
            uploaded = files.upload()

            if not uploaded:
                print("❌ 파일이 업로드되지 않았습니다.")
                return

            test_pdf_path = list(uploaded.keys())[0]
            print(f"✅ 업로드 완료: {test_pdf_path}")
        else:
            test_pdf_path = input("테스트 질문 PDF 파일 경로를 입력하세요: ").strip()
            if not os.path.exists(test_pdf_path):
                print("❌ 파일을 찾을 수 없습니다.")
                return

        # 5. PDF에서 질문 추출
        boundary_questions, gold_questions, trap_questions = extract_questions_from_pdf(test_pdf_path)

        if not (boundary_questions or gold_questions or trap_questions):
            print("❌ 질문을 추출할 수 없습니다.")
            return

        # 6. AI 정확도 평가 통합 테스트 실행
        runner = EnhancedVectorDBTestRunner(
            vectordb, boundary_questions, gold_questions, trap_questions, reference_content
        )
        results = runner.run_evaluation()

        # 7. 결과 저장
        print("\n💾 결과를 JSON으로 저장 중...")
        json_results = []
        for result in results:
            json_results.append({
                "test_id": result.test_id,
                "test_type": result.test_type,
                "question": result.question,
                "answer": result.answer,
                "overall_score": result.overall_score,
                "accuracy_score": result.accuracy_score,
                "vectordb_quality": result.vectordb_quality,
                "ai_accuracy_score": result.ai_accuracy_score,
                "ai_accuracy_grade": result.ai_accuracy_grade,
                "ai_detailed_feedback": result.ai_detailed_feedback,
                "pass_criteria": result.pass_criteria,
                "response_time": result.response_time,
                "source_documents_count": len(result.source_documents),
                "feedback": result.feedback
            })

        timestamp = int(time.time())
        filename = f"ai_enhanced_crossfit_results_{timestamp}.json"

        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(json_results, f, ensure_ascii=False, indent=2)

        print(f"✅ 결과가 {filename}에 저장되었습니다.")

        if COLAB_ENV:
            files.download(filename)
            print("📥 결과 파일이 다운로드되었습니다.")

    except Exception as e:
        print(f"❌ 시스템 오류: {str(e)}")
        import traceback
        traceback.print_exc()

# 실행
if __name__ == "__main__":
    main()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
✅ Google Colab 환경 감지
✅ PyPDF2 라이브러리 사용
🏋️ VectorDB + AI 정확도 평가 시스템
📄 실제 크로스핏 PDF 데이터 + AI 기반 정확도 평가
🚀 AI 정확도 평가 통합 크로스핏 챗봇 시스템
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ OpenAI API 키 설정 완료
✅ PDF 폴더 발견: /content/drive/MyDrive/크로스핏_가이드 (29개 파일)
⚡ 기존 VectorDB 발견


/tmp/ipython-input-2716109866.py:277: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


✅ VectorDB 로딩 완료! (총 2136개 문서)

📚 참고 문서 로드 중...
📖 PDF 파일 읽는 중: L1_Training_Guide_Korean.pdf
  📄 총 244페이지 감지
  📄 진행률: 5/244 페이지
  📄 진행률: 10/244 페이지
  📄 진행률: 15/244 페이지
  📄 진행률: 20/244 페이지
  📄 진행률: 25/244 페이지
  📄 진행률: 30/244 페이지
  📄 진행률: 35/244 페이지
  📄 진행률: 40/244 페이지
  📄 진행률: 45/244 페이지
  📄 진행률: 50/244 페이지
  📄 진행률: 55/244 페이지
  📄 진행률: 60/244 페이지
  📄 진행률: 65/244 페이지
  📄 진행률: 70/244 페이지
  📄 진행률: 75/244 페이지
  📄 진행률: 80/244 페이지
  📄 진행률: 85/244 페이지
  📄 진행률: 90/244 페이지
  📄 진행률: 95/244 페이지
  📄 진행률: 100/244 페이지
  📄 진행률: 105/244 페이지
  📄 진행률: 110/244 페이지
  📄 진행률: 115/244 페이지
  📄 진행률: 120/244 페이지
  📄 진행률: 125/244 페이지
  📄 진행률: 130/244 페이지
  📄 진행률: 135/244 페이지
  📄 진행률: 140/244 페이지
  📄 진행률: 145/244 페이지
  📄 진행률: 150/244 페이지
  📄 진행률: 155/244 페이지
  📄 진행률: 160/244 페이지
  📄 진행률: 165/244 페이지
  📄 진행률: 170/244 페이지
  📄 진행률: 175/244 페이지
  📄 진행률: 180/244 페이지
  📄 진행률: 185/244 페이지
  📄 진행률: 190/244 페이지
  📄 진행률: 195/244 페이지
  📄 진행률: 200/244 페이지
  📄 진행률: 205/244 페이지
  📄 진행률: 210/244 페이지
  📄 진행률: 215/244 페이지
  📄 진행률: 

Saving 골든-셋_-경계-셋_-함정-셋-질문-30개.pdf to 골든-셋_-경계-셋_-함정-셋-질문-30개.pdf
✅ 업로드 완료: 골든-셋_-경계-셋_-함정-셋-질문-30개.pdf
📖 PDF 파일 읽는 중: 골든-셋_-경계-셋_-함정-셋-질문-30개.pdf
  📄 총 13페이지 감지
  📄 진행률: 5/13 페이지
  📄 진행률: 10/13 페이지
✅ PyPDF2로 텍스트 추출 완료: 14,682자
✅ PDF 텍스트 추출 완료: 14,682자
✅ 경계셋 섹션 추출
✅ 골드셋 섹션 추출
✅ 함정셋 섹션 추출
  📝 경계셋 파싱 결과: 30개 질문
  📝 골드셋 파싱 결과: 30개 질문
  📝 함정셋 파싱 결과: 29개 질문
📊 추출 완료:
  - 경계셋: 30개
  - 골드셋: 30개
  - 함정셋: 29개
  - 총 질문: 89개
🔧 AI 정확도 평가 통합 크로스핏 챗봇 테스트 시스템 초기화
----------------------------------------------------------------------
✅ AI 정확도 평가자 초기화 완료
✅ AI 정확도 평가자 연동 완료
✅ 개선된 VectorDB 크로스핏 챗봇 준비 완료!
✅ AI 평가 통합 초기화 완료 - 총 89개 테스트 케이스
🏋️ AI 정확도 평가 통합 크로스핏 챗봇 평가 시작

[ 1/89] 골드 - GOLD-01
질문: 크로스핏의  벤치마크 운동이란 무엇인가요?
🔍 검색된 문서 수: 8
  📄 문서 1: 레벨1 훈련 가이드  방법론 계속 판권 소유 레벨1 훈련 가이드 | 81 / 242    6가지 벤치마크 운동 2004년 10월 최초 출간. 이 문서는 CrossFit 프로그램의 ...
  📄 문서 2: 운동에 대해 다시 브리핑할 때는 선수들에게 무슨 운동인지(동작 순서, 반복 형식, 라운드 또는 시간) 그리고 필요한 가동범위 기준에  대해 알려준다. 코치는 모든 단일 운동을 시작...
  📄 문서 3: 노인들에게도 CrossFit의 운동 방법을 테스트한 결과, 이러한 특별한  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 결과 파일이 다운로드되었습니다.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
